In [37]:
import pandas as pd
import json 
from copy import copy

df = pd.read_csv('../inference/ensemble.csv') # labeling을 위한 csv
with open("../../dataset/train_v3_f2.json", "r") as f:
    labeled = json.load(f)
train = copy(labeled)
df.head()

,Unnamed: 0,PredictionString,image_id
0,0,0 0.15688165 168.93044 425.67078 202.53456 452...,test/0000.jpg
1,1,0 0.7936861 637.57263 641.0707 686.15234 842.4...,test/0001.jpg
2,2,0 0.806641 880.000512 459.99974399999996 1024....,test/0002.jpg
3,3,0 0.031178754 121.46039 252.17184 934.196 815....,test/0003.jpg
4,4,0 0.80973613 442.07855 410.56204 651.9489 551....,test/0004.jpg


In [39]:
from collections import OrderedDict
import numpy

data = OrderedDict()

pseudo_annotations =[]
pseudo_images=[]
size = len(df['image_id'])

cnt = 0
for i in range(size):
    pseudo_images.append(
        {
            "width": 1024,
            "height": 1024,
            "file_name": df['image_id'][i],
            "license": "0",
            "url": "",
            "flickr_url" : None,
            "coco_url" : None,
            "date_captured": 0,
            "id": i + 4883
        }
    )
    pred= df['PredictionString'][i]
    bbox = pred.split()
    bbox = map(float, bbox)
    bbox = list(bbox)
    bbox = numpy.array(bbox).reshape(-1,6)
   
    
    for d in bbox:
        if d[1] > 0.5: # confidence threshold
            x_min, y_min, x_max, y_max = tuple(d[2:6])
            w = x_max-x_min
            h  = y_max - y_min
            cls = int(d[0])
            pseudo_annotations.append(
                {
                    "id": 23144 + cnt,
                    "category_id": cls,
                    "area": w * h,
                    "bbox": [x_min, y_min, w, h],
                    "iscrowd": 0,
                    "image_id": i+4883,
                }
            )
            cnt +=1

print(cnt)
data["images"] = pseudo_images
data["annotations"] = pseudo_annotations

25525


In [40]:
cls = ['General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
pseudo_categories =[]
for i in range(len(cls)):

    pseudo_categories.append(
        {
            "id": i,
            "name": cls[i],
            "supercategory": cls[i]
        }
    )

data["categories"] = pseudo_categories


In [42]:
print(len(train['images']), len(data['images']))
train['images'].extend(data['images'])
train['annotations'].extend(data['annotations'])
print(f"Dataset size after extend : {len(train['images'])}")

3923 4871


8794

In [44]:
with open('../../dataset/pseudofinal.json', 'w', encoding="utf-8") as f:
    json.dump(train, f, ensure_ascii=False, indent="\t")